In [1]:
import pandas as pd

df = pd.read_csv('../data/it_dataset.csv')
df = df[df['description'].apply(len)<2000]

df_train, df_test = df.iloc[:45000], df.iloc[45000:]

In [3]:
df_train['key_skills_list'] = df_train['key_skills'].str.split(';')

/tmp/ipykernel_401296/1578208901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['key_skills_list'] = df_train['key_skills'].str.split(';')


In [4]:
all_key_skills = df_train['key_skills_list'].apply(pd.Series).stack().reset_index(drop=True)
all_key_skills_counts = all_key_skills.value_counts()

In [26]:
all_key_skills_counts = all_key_skills_counts[all_key_skills_counts>10]

In [27]:
df_test.description.iloc[0]

'Обязанности:  анализ документации и работа со статистикой; разработка нагрузочных скриптов (Java или C#) ; запуск тестов и анализ результатов, написание отчетов и составление документации; А также: разработка заглушек и эмуляторов, разработка утилит для генерации данных, работа с базами данных.  Требования:  уверенное знание одного из языков программирование: C/C++, Java, C#, Phython; умение работать с базами данных на уровне написания простейших SQL-запросов; знание архитектур приложений (толстый/тонкий клиент); возможность работать полный рабочий день.  Условия:  работа в молодом активном коллективе; конкурентная заработная плата, оформление по ТК РФ; постоянное обучение внутри компании; возможность профессионального роста; участие в тестировании ПО в крупных российских банках.    Первый этап - собеседование по онлайн.'

In [28]:
from flashtext.keyword import KeywordProcessor
keyword_processor = KeywordProcessor()
for key_skill in all_key_skills_counts.index:
    keyword_processor.add_keyword(key_skill)
    
keywords_found = keyword_processor.extract_keywords('Обязанности:  анализ документации и работа со статистикой; разработка нагрузочных скриптов (Java или C#) ; запуск тестов и анализ результатов, написание отчетов и составление документации; А также: разработка заглушек и эмуляторов, разработка утилит для генерации данных, работа с базами данных.  Требования:  уверенное знание одного из языков программирование: C/C++, Java, C#, Phython; умение работать с базами данных на уровне написания простейших SQL-запросов; знание архитектур приложений (толстый/тонкий клиент); возможность работать полный рабочий день.  Условия:  работа в молодом активном коллективе; конкурентная заработная плата, оформление по ТК РФ; постоянное обучение внутри компании; возможность профессионального роста; участие в тестировании ПО в крупных российских банках.    Первый этап - собеседование по онлайн.')
keywords_found

['С',
 'Анализ',
 'С',
 'С',
 'С',
 'Разработка',
 'С',
 'Java',
 'C#',
 'С',
 'С',
 'Анализ',
 'С',
 'С',
 'С',
 'Разработка',
 'Разработка',
 'ИТ',
 'Работа с базами данных',
 'программирование',
 'C/C++',
 'Java',
 'C#',
 'С',
 'С',
 'С',
 'SQL',
 'С',
 'ИТ',
 'С',
 'С',
 'ПО',
 'С',
 'ПО',
 'ПО',
 'Обучение',
 'С',
 'С',
 'С',
 'С',
 'С',
 'ПО',
 'С',
 'С',
 'С',
 'С',
 'ПО']

In [32]:
df_test['extracted_key_skills'] = df_test['description'].apply(lambda x: ';'.join(list(set(keyword_processor.extract_keywords(x)))))

/tmp/ipykernel_401296/3355085020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['extracted_key_skills'] = df_test['description'].apply(lambda x: ';'.join(list(set(keyword_processor.extract_keywords(x)))))


In [33]:
df_test['extracted_key_skills']

73582    Java;С;Разработка;программирование;C#;Обучение...
73583    Коммуникация;Грамотная устная и письменная реч...
73585    Битрикс;С;удаленная работа;Разработка;1С: Упра...
73588    Работа в команде;С;Разработка;удаленно;COM;HTT...
73589    Планирование работ;Достижение целей;Developmen...
                               ...                        
83362    ПК;Грамотная устная и письменная речь;ГИС;С;Ау...
83368    Нефтегаз;Стремление;B2B;Навыки продаж;С;Гибкос...
83371    Работа в команде;разработчик;ООП;Node.js;С;Диз...
83373                     С;Компьютер;Ethernet;FI;ПО;ИТ;1с
83374    MPLS;С;Технолог;Телеком;VoIP;Навыки ведения пе...
Name: extracted_key_skills, Length: 5705, dtype: object

In [35]:
df_test.to_csv('../data/it_dataset_test_search.csv', index = False)